Dataset was taken from http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('data/boston.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


This is the regression task and our target variable is medv

In [3]:
print(df.shape)
print(df.info)

(506, 14)
<bound method DataFrame.info of         crim    zn  indus  chas    nox     rm   age     dis  rad  tax  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273   
502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273   
503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273   
504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273   
505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273   

     ptratio   black  lstat  medv  
0       15.3  396.90   4.98  24.0  
1       1

In [6]:
#Splitting dataset into train and test partX_train, X_test, y_train, y_test = train_test_split(df.drop(['medv'], axis=1), df['medv'],
                                                    test_size=0.2, random_state=2021)
print("X_train shape:",X_train.shape)
print("y_train shape:",y_train.shape)

print("X_test shape:",X_test.shape)
print("y_test shape:",y_test.shape)

X_train shape: (404, 13)
y_train shape: (404,)
X_test shape: (102, 13)
y_test shape: (102,)


In [8]:
#Scaling featuresscaler = StandardScaler().fit(X_train)
#Ridge Regression will be utilizedmodel_ridge = Ridge()

For selecting the optimal alpha for Ridge Regression, the model will be trained on multiple alphas and then the best will be selected.

In [10]:
Dictionary for storing resultsridge_cv = dict()
#Defining the array of alphas, from which the optimal will be selectedalphas = 10**np.linspace(10,-2,500)*0.5

for a in alphas:
    model_ridge.set_params(alpha=a)
    ridge_cv[a] = cross_val_score(model_ridge, scaler.transform(X_train), y_train, 
                                  cv=5, scoring='neg_mean_squared_error').mean() * (-1)

In [14]:
results_2_1 = pd.DataFrame({'Alpha': ridge_cv.keys(), 'MSE': ridge_cv.values()})
#results_2_1.sort_values(by='MSE', ascending=True, inplace=True)
#results_2_1.reset_index(drop=True, inplace=True)
results_2_1

,Alpha,MSE
0,5.000000e+09,90.242284
1,4.730662e+09,90.242282
2,4.475832e+09,90.242280
3,4.234730e+09,90.242278
4,4.006615e+09,90.242276
...,...,...
495,6.239681e-03,23.526018
496,5.903564e-03,23.526027
497,5.585553e-03,23.526035
498,5.284673e-03,23.526042


In [15]:
optimal_alpha = results_2_1.Alpha[0]
optimal_alpha

5000000000.0

In [17]:
#Setting optimal alpha and training the modelmodel_ridge.set_params(alpha=optimal_alpha)
model_ridge.fit(scaler.transform(X_train), y_train)

Dictionary for storing resultsmse=dict()
#Storing test and treain MSE in dataframemse['Train'] = mean_squared_error(y_train, model_ridge.predict(scaler.transform(X_train))) 
mse['Test'] = mean_squared_error(y_test, model_ridge.predict(scaler.transform(X_test)))
results = pd.DataFrame({'MSE': mse.values()}, index=mse.keys())
results

,MSE
Train,89.605031
Test,63.881971


In [18]:
#Looking at coefficientscoefs = pd.DataFrame({'Parameters': X_train.columns, 'Coefficients':model_ridge.coef_})
coefs

,Parameters,Coefficients
0,crim,-2.956667e-07
1,zn,2.859463e-07
2,indus,-3.708673e-07
3,chas,1.682661e-07
4,nox,-3.313572e-07
5,rm,5.438124e-07
6,age,-2.898456e-07
7,dis,1.924502e-07
8,rad,-2.795177e-07
9,tax,-3.560235e-07
